In [1254]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

In [1255]:
#import CSV 
df = pd.read_csv('dataAnalystChallenge_people_2017.csv', parse_dates=[0])

In [1256]:
### birthday fields 
#drop incorrectly formatted birthday
df['birthday']=np.where(df['person_id_random'].isin(['7955562566', '5801463353']), '', df['birthday'])
#Convert birthday, signup_date fields to datetime
df['birthday'] = pd.to_datetime(df['birthday'])
df['signup_date'] = pd.to_datetime(df['signup_date'])
#Calculate age 
df['Age']=df['signup_date'].dt.year-df['birthday'].dt.year
#Drop original birthday field 
df=df.drop(['birthday', 'signup_date'],1)
#Drop Age field where user is less than 16 at the date of sign-up
df['Age'] = np.where(df['Age']<16, np.NaN, df['Age'])
# Fill N/A Age with average of available data 
df['Age']=df['Age'].fillna(df['Age'].mean())


In [1257]:
###gender fields
#classify missing data as 'male' - only  a handful of missing values, and over 70% of data set is male 
df['gender'] = np.where(df['gender']=='F',1, 0)

In [1258]:
###facebook friends fieds 
# Fill N/A fb_friend_count with average of available data 
df['fb_friend_count']=df['fb_friend_count'].fillna(df['fb_friend_count'].mean())


In [1259]:
### living arrangements fields
# convert current_home_years into a numerical variable 
df['current_home_years'] = df['current_home_years'].str[:1]
#Drop original current_home_months field 
df=df.drop(['current_home_months'],1)

# Fill N/A current_home_years with average of available data 
df['current_home_years']=df['current_home_years'].astype(np.float)
df['current_home_years']=df['current_home_years'].fillna(df['current_home_years'].mean())

# classify living situation as own -1 or rent - 0, all other cases classified as 'rent' 
df['living_situation'] = np.where (df['living_situation'] == 'own_home',1,0)

In [1260]:
### working arrangements fields
# convert current_job_years into a numerical variable
df['current_job_years'] = df['current_job_years'].str[:1]
df['current_job_years']=df['current_job_years'].astype(np.float)
df['current_job_months'] = np.where(df['current_job_years']>0,0,df['current_job_months'])
# re-create 'is_employed' job status, employed - 1, not employed -2 
df['job_status'] = np.where((df['current_job_months']>0)|(df['current_job_years']>0), 1,0)
# job years - 0 if none, 1 if under one year, etc. 
df['current_job_years']=np.where((df['job_status']<1),0,
                np.where((df['current_job_years']>0),df['current_job_years']+1,0+1))
df=df.drop(['current_job_months', 'is_employed'],1)

In [1261]:
###Education fields
#primary-0, high-school - 1, college -2, masters -3; none -> primary 
df['education']=np.where(df['education']=='masters','3',
                        np.where (df['education']=='college','2',
                            np.where (df['education']=='high_school','1',0)))

In [1262]:
### Income fields

df['how_often_paid'] = np.where((df['how_often_paid']=='daily')|(df['how_often_paid']=='weekly')|(df['how_often_paid']=='monthly'),
                                df['how_often_paid'], np.NaN)
df['how_much_paid'] = np.where(df['how_often_paid'].notnull(),df['how_much_paid'],0 )

df['how_often_paid'] = np.where((df['how_much_paid'].isnull())| (df['how_much_paid']==0),'',df['how_often_paid'])
# assume 11 month - 48 week - 5 day work year 
df['how_often_paid'] = np.where(df['how_often_paid']=='daily', 240, 
                               np.where(df['how_often_paid']=='weekly', 48, 
                                       np.where(df['how_often_paid']=='monthly',11,0)))
df['annual_wages'] = df['how_often_paid']*df['how_much_paid']
df['annual_wages'] = np.where((df['annual_wages']<df['annual_wages'].quantile(.05))|
                              (df['annual_wages']>df['annual_wages'].quantile(.95)), np.NaN, df['annual_wages'])

df['annual_wages']=df['annual_wages'].fillna(df['annual_wages'].mean())
df=df.drop(['how_often_paid','how_much_paid' ],1)

In [1263]:
###other data points fields
#drop referral source - insufficient data difficult to interpret 
df=df.drop(['referral_source'],1)
df['outstanding_loan']= np.where(df['outstanding_loan']=='yes',1,0)
# convert use_mobile_wallet_how_often & relationship_status to numerical categories
df['use_mobile_wallet_how_often']=np.where(df['use_mobile_wallet_how_often']=='10+','3',
                        np.where (df['use_mobile_wallet_how_often']=='daily','2',
                            np.where (df['use_mobile_wallet_how_often']=='few_times_a_week','1',0)))

df['relationship_status']=np.where(df['relationship_status']=='alternative','3',
                        np.where (df['relationship_status']=='long-term','2',
                            np.where (df['relationship_status']=='married','1',0)))

In [1264]:
df.to_csv('people_clean.csv', sep=',', index = False)
